<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#What-this-notebook-does" data-toc-modified-id="What-this-notebook-does-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What this notebook does</a></span></li><li><span><a href="#Usual-Firedrake-and-Jupyter-preliminaries" data-toc-modified-id="Usual-Firedrake-and-Jupyter-preliminaries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Usual Firedrake and Jupyter preliminaries</a></span></li><li><span><a href="#Problem-Data" data-toc-modified-id="Problem-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Problem Data</a></span></li><li><span><a href="#The-mesh-and-FEM-space" data-toc-modified-id="The-mesh-and-FEM-space-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>The mesh and FEM space</a></span></li><li><span><a href="#Plot-solutions" data-toc-modified-id="Plot-solutions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plot solutions</a></span></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Exercises</a></span></li></ul></div>

# Example 7: A coupled system of two equations

[Niall Madden](https://www.niallmadden.ie/) _Feb 2025_

This notebook discusses how to solve a coupled system of two equations.



## What this notebook does 
We'll consider methods for solving Poisson's Equation: _find $u_1(x)$ and $u_2(x)$ such that_
$$-\varepsilon_1 u_1''(x) + a_{11} u_1(x) + a_{12} u_2(x) = f_1(x),$$
$$-\varepsilon_2 u_2''(x) + a_{21} u_1(x) + a_{22} u_2(x) = f_2(x),$$
for $x \in \Omega :=(0,1),$$
and subject to homogeneous Dirichlet boundary condition.

The specific example we'll study is:
$$-(0.1)u_1''(x) + 2 u_1(x) - u_2(x) = 1,$$
$$-(0.01)u_2''(x) - 2 u_1(x) + 3u_2(x) = 2,$$
for $x \in \Omega :=(0,1),$$
and subject to homogeneous Dirichlet boundary condition.

## Usual Firedrake and Jupyter preliminaries

In [ ]:
import os
if "OMP_NUM_THREADS" not in os.environ:
    os.environ["OMP_NUM_THREADS"] = "1"  
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt

## Problem Data

In [ ]:
epsilon1,epsilon2  = Constant(0.01), Constant(.01)
a11,a12 = Constant(2.0), Constant(-1.0)
a21,a22 = Constant(-2.0), Constant(3.0)
f1,f2 = Constant(2.0), Constant(3.0)

## The mesh and FEM space

In [ ]:
N = 64   # number of mesh intervals in space
mesh1D = UnitIntervalMesh(N)

To create our FEM space, we'll first define a scalar space, $V$, and then $W=V^2$.

In [ ]:
Ue = FiniteElement("CG", mesh1D.ufl_cell(), 1)
V = FunctionSpace(mesh1D, Ue)
W = V*V

Now create a pair of trail and test functions:

In [ ]:
u1,u2 = TrialFunction(W)    
v1,v2 = TestFunction(W)

The bilinear form is 
$$
a = \varepsilon_1 (u_1',v_1') + (a_{11}u_1 + a_{12}u_2, v_1) + \varepsilon_2 (u_2',v_2') + (a_{21}u_1 + a_{22}u_2, v_2).
$$
The linear form is 
$$ l = (f_1,v_1) + (f_2,v_2)$$.

In [ ]:
a =   epsilon1*u1.dx(0)*v1.dx(0)*dx + (a11*u1 + a12*u2)*v1*dx
a +=  epsilon2*u2.dx(0)*v2.dx(0)*dx + (a21*u1 + a22*u2)*v2*dx
L = f1*v1*dx  + f2*v2*dx  

There is a minor technicality on how the boundary conditions are implemented. We need to apply them to each component, using the `sub()` method. 

Then we solve. And then use `subfunctions` to extract the components.

In [ ]:
a =   epsilon1*u1.dx(0)*v1.dx(0)*dx + (a11*u1 + a12*u2)*v1*dx
a +=  epsilon2*u2.dx(0)*v2.dx(0)*dx + (a21*u1 + a22*u2)*v2*dx
L = f1*v1*dx  + f2*v2*dx  
    
BC1 = DirichletBC(W.sub(0), 0.0, "on_boundary")
BC2 = DirichletBC(W.sub(1), 0.0, "on_boundary")

UN = Function(W)
#prob = LinearVariationalProblem(a, L, UN, bcs=[BC1,BC2])
#solver =  LinearVariationalSolver(prob)
#solver.solve()

solve(a == L, UN, bcs=[BC1,BC2])

uN1, uN2 = UN.subfunctions

## Plot solutions

In [ ]:
fig, axes = plt.subplots()
plot(uN1, axes=axes, label='$u_1$')
plot(uN2, axes=axes, label='$u_2$')
# x_vals = mesh1D.coordinates.dat.data[:]
# plt.plot(x_vals, u1.dat.data[:],'r--o', label='$u^N$')  # use for small N
plt.xlabel("x")
plt.legend()

##  Exercises

1. Re-do this for a two-dimensional problem